In [18]:
from dataclasses import dataclass

from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler


@dataclass
class MyMessageType:
    content: str


class MyAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("MyAgent")

    @message_handler
    async def handle_my_message_type(self, message: MyMessageType, ctx: MessageContext) -> None:
        print(f"{self.id.type} received message: {message.content}")

In [19]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
load_dotenv()

class MyAssistant(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o")
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: MyMessageType, ctx: MessageContext) -> None:
        print(f"{self.id.type} received message: {message.content}")
        response = await self._delegate.on_messages(
            [TextMessage(content=message.content, source="user")], ctx.cancellation_token
        )
        # create a text file and save the response
        with open(f"{self.id.type}_response.txt", "w") as f:
            f.write(response.chat_message.content)
        print(f"{self.id.type} responded: {response.chat_message.content}")

In [20]:
from autogen_core import SingleThreadedAgentRuntime

runtime = SingleThreadedAgentRuntime()
await MyAgent.register(runtime, "my_agent", lambda: MyAgent())
await MyAssistant.register(runtime, "my_assistant", lambda: MyAssistant("my_assistant"))

AgentType(type='my_assistant')

In [21]:
runtime.start()
await runtime.send_message(MyMessageType("Hello from MyAgent!"), AgentId(type="my_agent", key="default"))
await runtime.send_message(MyMessageType("Hello from MyAssistant!"), AgentId(type="my_assistant", key="default"))
await runtime.stop()

my_agent received message: Hello from MyAgent!
my_assistant received message: Hello from MyAssistant!
my_assistant responded: Hello! How can I assist you today?
